In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pathlib import Path
import xarray as xr
import tensorflow as tf
from shared.models import SAT1Start, ShallowConvNet, EEGNet
from shared.generators import SAT1DataGenerator
from shared.utilities import earlyStopping_cb
from sklearn.metrics import classification_report

%env TF_FORCE_GPU_ALLOW_GROWTH=true
%env TF_GPU_ALLOCATOR=cuda_malloc_async

2023-08-22 13:17:34.056260: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-22 13:17:34.871297: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


env: TF_FORCE_GPU_ALLOW_GROWTH=true
env: TF_GPU_ALLOCATOR=cuda_malloc_async


### Set up data

In [2]:
data_path = Path("data/sat1/split_stage_data.nc")

data = xr.load_dataset(data_path)

In [3]:
train_data = data.sel(
    participant=[
        "0021",
        "0022",
        "0023",
        "0024",
    ]
)
test_data = data.sel(participant=["0025"])

In [4]:
batch_size = 32
train_gen = SAT1DataGenerator(train_data, batch_size=batch_size)
test_gen = SAT1DataGenerator(test_data, batch_size=batch_size)

# tf.config.optimizer.set_experimental_options({"layout_optimizer": False})

In [11]:
# Run before re-training to clear up VRAM
import gc

gc.collect()
tf.keras.backend.clear_session()
del model

NameError: name 'model' is not defined

In [5]:
model = SAT1Start(30, 157, 4)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 30, 157, 1)]      0         
                                                                 
 conv2d (Conv2D)             (None, 30, 153, 16)       96        
                                                                 
 dropout (Dropout)           (None, 30, 153, 16)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 149, 32)       12832     
                                                                 
 dropout_1 (Dropout)         (None, 26, 149, 32)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 22, 149, 64)       10304     
                                                                 
 batch_normalization (Batch  (None, 22, 149, 64)       256   

2023-08-22 13:17:50.336489: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-22 13:17:50.435901: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-22 13:17:50.435983: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-22 13:17:50.439112: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-08-22 13:17:50.439171: I tensorflow/compile

In [6]:
model.fit(
    train_gen,
    epochs=20,
    # TODO: Create val_gen (use 6th participant?)
    validation_data=test_gen,
    batch_size=batch_size,
)

Epoch 1/20


2023-08-22 13:17:55.858466: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-08-22 13:17:58.049042: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-22 13:17:59.209554: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-08-22 13:17:59.623673: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-08-22 13:17:59.633651: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fd664043750 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-22 13:17:59.633677: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0

100/100 [==============================] - 118s 1s/step - loss: 1.5412 - accuracy: 0.2584 - val_loss: 1.4398 - val_accuracy: 0.2500
Epoch 2/20
100/100 [==============================] - 114s 1s/step - loss: 1.5036 - accuracy: 0.2844 - val_loss: 1.4643 - val_accuracy: 0.2688
Epoch 3/20
100/100 [==============================] - 115s 1s/step - loss: 1.4865 - accuracy: 0.2941 - val_loss: 1.5363 - val_accuracy: 0.2500
Epoch 4/20
100/100 [==============================] - 113s 1s/step - loss: 1.4356 - accuracy: 0.3200 - val_loss: 1.6311 - val_accuracy: 0.2500
Epoch 5/20
100/100 [==============================] - 115s 1s/step - loss: 1.4058 - accuracy: 0.3459 - val_loss: 1.6917 - val_accuracy: 0.2575
Epoch 6/20
100/100 [==============================] - 113s 1s/step - loss: 1.3562 - accuracy: 0.3716 - val_loss: 1.9300 - val_accuracy: 0.2575
Epoch 7/20
100/100 [==============================] - 116s 1s/step - loss: 1.3139 - accuracy: 0.3925 - val_loss: 2.4698 - val_accuracy: 0.2575
Epoch 8/20